In [1]:
import requests 

In [2]:
import docx
import spacy
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load the Ruberta model and tokenizer from Hugging Face transformers
model_name = "cointegrated/rubert-tiny2"  # Pre-trained Ruberta model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)
    
    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    
    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunk_and_encode_document(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Encode each chunk into vectors
    encoded_chunks = []

    for chunk in chunks:
        # Tokenize the chunk
        inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)

        # Pass the input through the model and get the hidden states
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (CLS token embedding)
        embeddings = outputs.last_hidden_state[:, 0, :]

        # Normalize the embeddings
        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        # Append the normalized embedding to the list
        encoded_chunks.append(normalized_embeddings)

    return chunks, encoded_chunks


file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks, encoded_chunks = chunk_and_encode_document(document_text)

In [3]:
def encode_query(query_text):
    # Encode the query text into a vector
    inputs = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (CLS token embedding)
    query_embedding = outputs.last_hidden_state[:, 0, :]

    # Normalize the embedding
    normalized_query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)

    return normalized_query_embedding



def vector_search(query_embedding, encoded_chunks, chunks):
    # Compute cosine similarity between query embedding and all chunk embeddings
    similarities = []

    for emb in encoded_chunks:
        sim = cosine_similarity(query_embedding.numpy().reshape(1, -1), emb.numpy().reshape(1, -1))
        similarities.append(sim.item())

    # Find indices of the top 3 most similar chunks
    closest_indices = np.argsort(similarities)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks
query = 'Что такое ПУД?'
prompt = {
    "modelUri": "gpt://folder id/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "system",
            "text": "ты умный помошник студента, постарайся ответить на поставленный вопрос"
        },
        {
            "role": "user",
            "text": query
        }
    ]
}


url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key your key"
}

response = requests.post(url, headers=headers, json=prompt)
result = response.text
response_data = response.json()
assistant_text = response_data['result']['alternatives'][0]['message']['text']
print(assistant_text)



**ПУД** — вероятно, в данном контексте имеется в виду устаревшая русская мера веса, равная четырём килограммам. Однако аббревиатура «ПУД» может иметь и другое значение.

В современном мире ПУДом называют персональный уровень достижений (англ. personal level of achievements, сокр. PUL), который используется для оценки успеваемости студентов некоторых учебных заведений. ПУД отражает совокупность личных достижений студента в различных областях и является одним из критериев его итоговой оценки.

Чтобы определить персональный уровень достижений, учебные заведения могут устанавливать различные критерии, такие как участие в активных мероприятиях, научные исследования, спортивные достижения, творческие работы, результаты экзаменов и другие. ПУД может включать в себя как количественные, так и качественные показатели, которые оцениваются по определённой методике.

Таким образом, аббревиатура ПУД может означать персональный уровень достижений.


In [4]:
query = query + assistant_text
query_embedding = encode_query(query)
closest_chunks = vector_search(query_embedding, encoded_chunks, chunks)
i=1
print("Closest chunks to the query:")
for idx, chunk in enumerate(closest_chunks, start=1):
    print('chunk ',i)
    print()
    i+=1
    print(f"{idx}. {chunk}")
    print()

Closest chunks to the query:
chunk  1

1. Элементам контроля . 
 Основополагающим принципом оценивания результатов обучения студентов является накопительный характер промежуточной аттестации : оценка по Дисциплине зависит от постоянных усилий студента , которые выявляются посредством каждого из Элементов контроля . 
 Оценка за Элемент контроля , объявленная преподавателем студенту в установленном Положением порядке , не может быть изменена , кроме случаев , предусмотренных Положением . 
 Студент обязан добросовестно осваивать образовательную программу и выполнять учебный план ( в том числе индивидуальный ) . 
 Добросовестное освоение образовательной программы студентом НИУ ВШЭ в течение учебного периода в том числе включает : выполнение учебного плана ; консультирование с преподавателями или учебными ассистентами ( при наличии у студента трудностей в освоении образовательной программы ) ; отсутствие академических задолженностей или наличие академических задолженностей не более чем по д

In [5]:
context = str(closest_chunks)

prompt = {
    "modelUri": "gpt://folder id/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "system",
            "text": "ты умный помошник студента, используй документы ниже, чтобы ответить на поставленный вопрос, если в документах этой информации нет, то так и скажи\
            вот ДОКУМЕНТЫ: " + context
        },
        {
            "role": "user",
            "text": query
        }
    ]
}


url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key your key"
}

response = requests.post(url, headers=headers, json=prompt)
result = response.text
response_data = response.json()
assistant_text = response_data['result']['alternatives'][0]['message']['text']
print(assistant_text)

На основании предоставленной информации можно сделать вывод, что **ПУД** – это **п** **ерсональный уровень достижений**.

**ПУД** может включать в себя **оценки по различным Дисциплинам**, оценки за которые рассчитываются по разным формулам и могут влиять на общую оценку по промежуточной аттестации. 

Также в ПУД могут входить и другие параметры, например, участие в научной работе, спортивные и творческие достижения, активность в жизни университета и прочее. Все они учитываются при расчёте персонального уровня достижений и влияют на общую оценку.
